In [1]:
import os
import json

import torch
import torch.nn as nn
from torch import optim
import numpy as np
import pandas as pd

import torch_utils
from torch import distributions

import gymnasium as gym
import gymnasium_robotics as gym_robo
import models
import cnn_models
import rl_agents
import rl_callbacks
import helper
import gym_helper
import wandb_support
import wandb
import gym_helper

# from mpi4py import MPI

In [ ]:
import mujoco

In [ ]:
mujoco.MjModel

In [ ]:
gym_robo.__version__

In [ ]:
def get_default_device():
    """Returns the default device for computations, GPU if available, otherwise CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')

device = get_default_device()
print(f"Using device: {device}")

# TEST

In [ ]:
gym_robo.register_robotics_envs()

In [ ]:
gym.envs.registration.registry

In [ ]:
wandb.login(key='758ac5ba01e12a3df504d2db2fec8ba4f391f7e6')

In [ ]:
env = gym.make('FetchPush-v2', max_episode_steps=100, render_mode='rgb_array')
env = gym.wrappers.RecordVideo(env, 'test/', episode_trigger=lambda i: i%1==0)

episodes = 10


for episode in range(episodes):
    done = False
    obs, _ = env.reset()
    while not done:
        obs, r, term, trunc, dict = env.step(env.action_space.sample())
        if term or trunc:
            done = True
env.close()

In [ ]:
env = gym.make("FetchReach-v2")
env.reset()
obs, reward, terminated, truncated, info = env.step(env.action_space.sample())

# The following always has to hold:
assert reward == env.compute_reward(obs["achieved_goal"], obs["desired_goal"], info)
assert truncated == env.compute_truncated(obs["achieved_goal"], obs["desired_goal"], info)
assert terminated == env.compute_terminated(obs["achieved_goal"], obs["desired_goal"], info)

In [ ]:
env.compute_reward()

In [ ]:
env = gym.make('FetchPush-v2', render_mode='rgb_array')

In [ ]:
if hasattr(env, "distance_threshold"):
    print('true')
else:
    print('false')

In [ ]:
if env.get_wrapper_attr("distance_threshold"):
    print('true')

In [ ]:
print(dir(env))


# DDPG

In [ ]:
env = gym.make('Pendulum-v1')

In [ ]:
# build actor

dense_layers = [
    (
        400,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        300,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    )
]

actor = models.ActorModel(env, cnn_model=None, dense_layers=dense_layers, optimizer='Adam',
                          optimizer_params={'weight_decay':0.01}, learning_rate=0.001, normalize_layers=True)

In [ ]:
actor

In [ ]:
ddpg_agent.actor_model

In [ ]:
ddpg_agent.target_actor_model

In [ ]:
# build critic

state_layers = [
    (
        400,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    )
]

merged_layers = [
    (
        300,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    )
]


critic = models.CriticModel(env=env, cnn_model=None, state_layers=state_layers, merged_layers=merged_layers,
                            optimizer='Adam', optimizer_params={'weight_decay':0.01}, learning_rate=0.002, normalize_layers=True)

In [ ]:
critic

In [ ]:
replay_buffer = helper.ReplayBuffer(env, 100000)
noise = helper.OUNoise(shape=env.action_space.shape, dt=1.0, device='cuda')

In [ ]:
ddpg_agent = rl_agents.DDPG(env=env,
                            actor_model=actor,
                            critic_model=critic,
                            discount=0.99,
                            tau=0.005,
                            replay_buffer=replay_buffer,
                            noise=noise,
                            callbacks=[rl_callbacks.WandbCallback('Pendulum-v1')])

In [ ]:
ddpg_agent.critic_model

In [ ]:
ddpg_agent.target_critic_model

In [ ]:
ddpg_agent.train(100, True, 10)

In [ ]:
ddpg_agent.test(10, True, 1)

# Actor Critic

In [ ]:
env = gym.make("CartPole-v1")

In [ ]:
dense_layers = [
    (128, 'relu', "kaiming normal"),
    (256, 'relu', "kaiming normal"),
    ]



In [ ]:
policy_model = models.PolicyModel(env=env, dense_layers=dense_layers, optimizer='Adam', learning_rate=0.001,)

In [ ]:
for param in policy_model.parameters():
    print(param)

In [ ]:
value_model = models.ValueModel(env, dense_layers=dense_layers, optimizer='Adam', learning_rate=0.001)

In [ ]:
value_model

In [ ]:
for params in value_model.parameters():
    print(params)

In [ ]:
actor_critic = rl_agents.ActorCritic(env,
                                     policy_model,
                                     value_model,
                                     discount=0.99,
                                     policy_trace_decay=0.5,
                                     value_trace_decay=0.5,
                                     callbacks=[rl_callbacks.WandbCallback('CartPole-v1-Actor-Critic')])

In [ ]:
actor_critic.train(200)

In [ ]:
actor_critic.test(10, True, 1)

# REINFORCE

In [ ]:
env = gym.make("CartPole-v1")

In [ ]:
dense_layers = [
    (128, 'relu', {
                    "kaiming normal": {
                        "a":1.0,
                        "mode":'fan_in'
                    }
                },
    ),
    # (256, 'relu', {
    #                 "kaiming_normal": {
    #                     "a":0.0,
    #                     "mode":'fan_in'
    #                 }
    #             },
    # )
    ]

In [ ]:
dense_layers = [(128, 'relu', "kaiming normal")]

In [ ]:
value_model = models.ValueModel(env, dense_layers, 'Adam', 0.001)

In [ ]:
for param in value_model.parameters():
    print(param)

In [ ]:
policy_model = models.PolicyModel(env, dense_layers, 'Adam', 0.001)

In [ ]:
for param in policy_model.parameters():
    print(param)

In [ ]:
reinforce = rl_agents.Reinforce(env, policy_model, value_model, 0.99, [rl_callbacks.WandbCallback('CartPole-v0_REINFORCE', chkpt_freq=100)])

In [ ]:
reinforce.train(200, True, 50)

In [ ]:
reinforce.test(10, True, 1)

# DDPG w/CNN

In [ ]:
env = gym.make('CarRacing-v2')

In [ ]:
cnn_layers = [
    # {
    #     "batchnorm":
    #     {
    #         "num_features":3
    #     }
    # },
    {
        "conv":
        {
            "out_channels": 32,
            "kernel_size": 7,
            "stride": 3,
            "padding": 'valid',
            "bias": False
        }
    },
    {
        "relu":
        {

        }
    },
    {
        "batchnorm":
        {
            "num_features":32
        }
    },
    {
        "conv":
        {
            "out_channels": 32,
            "kernel_size": 5,
            "stride": 3,
            "padding": 'valid',
            "bias": False,
        }
    },
    {
        "relu":
        {

        }
    },
    {
        "batchnorm":
        {
            "num_features":32
        }
    },
    {
        "conv":
        {
            "out_channels": 32,
            "kernel_size": 3,
            "stride": 3,
            "padding": 'valid',
            "bias": False,
        }
    },
]

In [ ]:
cnn = cnn_models.CNN(cnn_layers, env)

In [ ]:
cnn

In [ ]:
# build actor

dense_layers = [
    (
        64,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        64,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        64,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
]

actor = models.ActorModel(env, cnn_model=cnn, dense_layers=dense_layers, optimizer="Adam", optimizer_params={'weight_decay':0.0}, learning_rate=0.0001, normalize=False)

In [ ]:
actor

In [ ]:
# build critic

state_layers = [
    
]

merged_layers = [
    (
        64,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        64,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        64,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    )
]


critic = models.CriticModel(env=env, cnn_model=cnn, state_layers=state_layers, merged_layers=merged_layers, optimizer="Adam", optimizer_params={'weight_decay':0.0}, learning_rate=0.0001, normalize=False)

In [ ]:
critic

In [ ]:
replay_buffer = helper.ReplayBuffer(env, 1000000, goal_shape=(1,))
noise = helper.OUNoise(shape=env.action_space.shape, mean=0.0, theta=0.15, sigma=0.01, dt=1.0, device='cuda')

In [ ]:
ddpg_agent = rl_agents.DDPG(
    env,
    actor,
    critic,
    discount=0.98,
    tau=0.05,
    action_epsilon=0.2,
    replay_buffer=replay_buffer,
    batch_size=128,
    noise=noise,
    callbacks=[rl_callbacks.WandbCallback("CarRacing-v2")]
)

In [ ]:
ddpg_agent.train(1000, True, 10)

In [ ]:
wandb.finish()

In [ ]:
wandb.login()

# HER

In [ ]:
env = gym.make("Reacher-v4")

In [ ]:
_,_ = env.reset()

In [ ]:
achieved_goal = gym_helper.reacher_achieved_goal(env)
action = env.action_space.sample()
env.step(action)
print(f'observation: {env.get_wrapper_attr("_get_obs")()}')
print(f'distance to goal: {env.get_wrapper_attr("_get_obs")()[8::]}')
print(f'fingertip: {env.get_wrapper_attr("get_body_com")("fingertip")}')
print(f'target: {env.get_wrapper_attr("get_body_com")("target")}')

In [ ]:
next_achieved_goal = env.get_wrapper_attr("_get_obs")()[8::]
desired_goal = [0.0, 0.0, 0.0]

In [ ]:
reward_func(env, action, achieved_goal, next_achieved_goal, desired_goal, 0.05)

In [ ]:
desired_goal_func, achieved_goal_func, reward_func = gym_helper.get_her_goal_functions(env)

In [ ]:
desired_goal_func(env).shape

In [ ]:
# build actor

dense_layers = [
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
]

actor = models.ActorModel(env,
                          cnn_model=None,
                          dense_layers=dense_layers,
                          goal_shape=(3,),
                          optimizer="Adam",
                          optimizer_params={'weight_decay':0.0},
                          learning_rate=0.0001, normalize=False)

In [ ]:
# build critic

state_layers = [
    
]

merged_layers = [
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    )
]


critic = models.CriticModel(env=env,
                            cnn_model=None,
                            state_layers=state_layers,
                            merged_layers=merged_layers,
                            goal_shape=(3,),
                            optimizer="Adam",
                            optimizer_params={'weight_decay':0.0},
                            learning_rate=0.0001,
                            normalize=False)

In [ ]:
goal_shape = desired_goal_func(env).shape
replay_buffer = helper.ReplayBuffer(env, 100000, goal_shape)
# noise = helper.OUNoise(shape=env.action_space.shape,
#                        mean=0.0,
#                        theta=0.05,
#                        sigma=0.15,
#                        dt=1.0, device='cuda')

noise=helper.NormalNoise(shape=env.action_space.shape,
                         mean = 0.0,
                         stddev=0.05,
                         )

In [ ]:
ddpg_agent = rl_agents.DDPG(env=env,
                            actor_model=actor,
                            critic_model=critic,
                            discount=0.98,
                            tau=0.05,
                            action_epsilon=0.2,
                            replay_buffer=replay_buffer,
                            batch_size=256,
                            noise=noise,
                            callbacks=[rl_callbacks.WandbCallback('Reacher-v4')])

In [ ]:
her = rl_agents.HER(ddpg_agent,
                    strategy='future',
                    num_goals=4,
                    tolerance=0.001,
                    desired_goal=desired_goal_func,
                    achieved_goal=achieved_goal_func,
                    reward_fn=reward_func)

In [ ]:
her.train(10, 50, 16, 40, True, 1000)

In [ ]:
wandb.finish()

In [ ]:
her.test(10, True, 1)

In [ ]:
her.save()

In [ ]:
her.agent.goal_normalizer.running_std

In [ ]:
loaded_her = rl_agents.HER.load("/workspaces/RL_Agents/pytorch/src/app/assets/models/her")

In [ ]:
loaded_her.agent.replay_buffer.sample(10)

In [ ]:
loaded_her.agent.state_normalizer.running_cnt

In [ ]:
loaded_her.get_config()

In [ ]:
loaded_her.test(10, True, 1)

In [ ]:
10e4

# HER w/CNN

In [ ]:
env = gym.make('CarRacing-v2')

In [ ]:
_,_ = env.reset()

In [ ]:
desired_goal_func, achieved_goal_func, reward_func = gym_helper.get_her_goal_functions(env)

In [ ]:
desired_goal(env).shape

In [ ]:
cnn_layers = [
    # {
    #     "batchnorm":
    #     {
    #         "num_features":3
    #     }
    # },
    {
        "conv":
        {
            "out_channels": 32,
            "kernel_size": 7,
            "stride": 3,
            "padding": 'valid',
            "bias": False
        }
    },
    {
        "relu":
        {

        }
    },
    {
        "batchnorm":
        {
            "num_features":32
        }
    },
    {
        "conv":
        {
            "out_channels": 32,
            "kernel_size": 5,
            "stride": 3,
            "padding": 'valid',
            "bias": False,
        }
    },
    {
        "relu":
        {

        }
    },
    {
        "batchnorm":
        {
            "num_features":32
        }
    },
    {
        "conv":
        {
            "out_channels": 32,
            "kernel_size": 3,
            "stride": 3,
            "padding": 'valid',
            "bias": False,
        }
    },
]

cnn = cnn_models.CNN(cnn_layers, env)

In [ ]:
# build actor

dense_layers = [
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
]

actor = models.ActorModel(env,
                          cnn_model=cnn,
                          dense_layers=dense_layers,
                          goal_shape=(1,),
                          optimizer="Adam",
                          optimizer_params={'weight_decay':0.0},
                          learning_rate=0.001, normalize=False)

In [ ]:
actor

In [ ]:
# build critic

state_layers = [
    
]

merged_layers = [
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        256,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    )
]


critic = models.CriticModel(env=env,
                            cnn_model=cnn,
                            state_layers=state_layers,
                            merged_layers=merged_layers,
                            goal_shape=(1,),
                            optimizer="Adam",
                            optimizer_params={'weight_decay':0.0},
                            learning_rate=0.001,
                            normalize=False)

In [ ]:
critic

In [ ]:
goal_shape = desired_goal_func(env).shape
replay_buffer = helper.ReplayBuffer(env, 100000, goal_shape)
# noise = helper.OUNoise(shape=env.action_space.shape,
#                        mean=0.0,
#                        theta=0.05,
#                        sigma=0.15,
#                        dt=1.0, device='cuda')

noise=helper.NormalNoise(shape=env.action_space.shape,
                         mean = 0.0,
                         stddev=0.05,
                         )

In [ ]:
ddpg_agent = rl_agents.DDPG(env=env,
                            actor_model=actor,
                            critic_model=critic,
                            discount=0.98,
                            tau=0.05,
                            action_epsilon=0.2,
                            replay_buffer=replay_buffer,
                            batch_size=256,
                            noise=noise,
                            callbacks=[rl_callbacks.WandbCallback('CarRacing-v2')])

In [ ]:
ddpg_agent.actor_model

In [ ]:
her = rl_agents.HER(ddpg_agent,
                    strategy='future',
                    num_goals=4,
                    tolerance=1,
                    desired_goal=desired_goal_func,
                    achieved_goal=achieved_goal_func,
                    reward_fn=reward_func)

In [ ]:
her.agent.actor_model

In [ ]:
her.train(num_epochs=20,
          num_cycles=50,
          num_episodes=16,
          num_updates=40,
          render=True,
          render_freq=20
        )

In [ ]:
her = rl_agents.HER.load("/workspaces/RL_Agents/pytorch/src/app/models/her")

In [ ]:
wandb.finish()

In [ ]:
# reset environment
state, _ = her.agent.env.reset()
# instantiate empty lists to store current episode trajectory
states, actions, next_states, dones, state_achieved_goals, \
next_state_achieved_goals, desired_goals = [], [], [], [], [], [], []
# set desired goal
desired_goal = her.desired_goal_func(her.agent.env)
# set achieved goal
state_achieved_goal = her.achieved_goal_func(her.agent.env)
# add initial state and goals to local normalizer stats
her.state_normalizer.update_local_stats(state)
her.goal_normalizer.update_local_stats(desired_goal)
her.goal_normalizer.update_local_stats(state_achieved_goal)
# set done flag
done = False
# reset episode reward to 0
episode_reward = 0
# reset steps counter for the episode
episode_steps = 0

while not done:
    # get normalized values for state and desired goal
    state_norm = her.state_normalizer.normalize(state)
    desired_goal_norm = her.goal_normalizer.normalize(desired_goal)
    # get action
    action = her.agent.get_action(state_norm, desired_goal_norm, grad=False)
    # take action
    next_state, reward, term, trunc, _ = her.agent.env.step(action)
    # get next state achieved goal
    next_state_achieved_goal = her.achieved_goal_func(her.agent.env)
    # add next state and next state achieved goal to normalizers
    her.state_normalizer.update_local_stats(next_state)
    her.goal_normalizer.update_local_stats(next_state_achieved_goal)
    # store trajectory in replay buffer (non normalized!)
    her.agent.replay_buffer.add(state, action, reward, next_state, done,\
                                    state_achieved_goal, next_state_achieved_goal, desired_goal)
    
    # append step state, action, next state, and goals to respective lists
    states.append(state)
    actions.append(action)
    next_states.append(next_state)
    dones.append(done)
    state_achieved_goals.append(state_achieved_goal)
    next_state_achieved_goals.append(next_state_achieved_goal)
    desired_goals.append(desired_goal)

    # add to episode reward and increment steps counter
    episode_reward += reward
    episode_steps += 1
    # update state and state achieved goal
    state = next_state
    state_achieved_goal = next_state_achieved_goal
    # update done flag
    if term or trunc:
        done = True

In [ ]:
# package episode states, actions, next states, and goals into trajectory tuple
trajectory = (states, actions, next_states, dones, state_achieved_goals, next_state_achieved_goals, desired_goals)

In [ ]:
states, actions, next_states, dones, state_achieved_goals, next_state_achieved_goals, desired_goals = trajectory

In [ ]:
for idx, (s, a, ns, d, sag, nsag, dg) in enumerate(zip(states, actions, next_states, dones, state_achieved_goals, next_state_achieved_goals, desired_goals)):
    print(f'a={a}, d={d}, sag={sag}, nsag={nsag}, dg={dg}')

In [ ]:
strategy = "future"
num_goals = 4

# loop over each step in the trajectory to set new achieved goals, calculate new reward, and save to replay buffer
for idx, (state, action, next_state, done, state_achieved_goal, next_state_achieved_goal, desired_goal) in enumerate(zip(states, actions, next_states, dones, state_achieved_goals, next_state_achieved_goals, desired_goals)):

    if strategy == "final":
        new_desired_goal = next_state_achieved_goals[-1]
        new_reward = her.reward_fn(state_achieved_goal, next_state_achieved_goal, new_desired_goal)
        print(f'transition: action={action}, reward={new_reward}, done={done}, state_achieved_goal={state_achieved_goal}, next_state_achieved_goal={next_state_achieved_goal}, desired_goal={new_desired_goal}')
        her.agent.replay_buffer.add(state, action, new_reward, next_state, done, state_achieved_goal, next_state_achieved_goal, new_desired_goal)

    if strategy == 'future':
        for i in range(num_goals):
            if idx + i + 1 >= len(states):
                break
            goal_idx = np.random.randint(idx + 1, len(states))
            new_desired_goal = next_state_achieved_goals[goal_idx]
            new_reward = her.reward_fn(state_achieved_goal, next_state_achieved_goal, new_desired_goal)
            print(f'transition: action={action}, reward={new_reward}, done={done}, state_achieved_goal={state_achieved_goal}, next_state_achieved_goal={next_state_achieved_goal}, desired_goal={new_desired_goal}')
            her.agent.replay_buffer.add(state, action, new_reward, next_state, done, state_achieved_goal, next_state_achieved_goal, new_desired_goal)
    

    


In [ ]:
s, a, r, ns, d, sag, nsag, dg = her.agent.replay_buffer.sample(100)

In [ ]:
for i in range(100):
    print(f'{i}: a={a[i]}, r={r[i]}, d={d[i]}, sag={sag[i]}, nsag={nsag[i]}, dg={dg[i]} ')

# HER Pendulum

In [ ]:
env = gym.make('Pendulum-v1')

In [ ]:
# build actor

dense_layers = [
    (
        400,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    ),
    (
        300,
        "relu",
        {
            "variance scaling": {
                "scale": 1.0,
                "mode": "fan_in",
                "distribution": "uniform",
            }
        },
    )
]

actor = models.ActorModel(env, cnn_model=None, dense_layers=dense_layers, optimizer='Adam',
                          optimizer_params={'weight_decay':0.01}, learning_rate=0.001, normalize=False)

In [ ]:
# build critic

state_layers = [
    
]

merged_layers = [
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    )
]


critic = models.CriticModel(env=env, cnn_model=None, state_layers=state_layers, merged_layers=merged_layers, optimizer="Adam", optimizer_params={'weight_decay':0.0}, learning_rate=0.001, normalize=False)

In [ ]:
replay_buffer = helper.ReplayBuffer(env, 100000, (3,))
noise = helper.OUNoise(shape=env.action_space.shape, dt=1.0, device='cuda')

In [ ]:
ddpg_agent = rl_agents.DDPG(env=env,
                            actor_model=actor,
                            critic_model=critic,
                            discount=0.99,
                            tau=0.005,
                            replay_buffer=replay_buffer,
                            noise=noise,
                            callbacks=[rl_callbacks.WandbCallback('Pendulum-v1')])

In [ ]:
def desired_goal_func(env):
    return np.array([0.0, 0.0, 0.0])

def achieved_goal_func(env):
    return env.get_wrapper_attr('_get_obs')()

def reward_func(env):
    pass

In [ ]:
her = rl_agents.HER(
    agent=ddpg_agent,
    strategy='none',
    desired_goal=desired_goal_func,
    achieved_goal=achieved_goal_func,
    reward_fn=reward_func,
    normalizer_clip=10.0
)

In [ ]:
her.agent.critic_model

In [ ]:
her.agent.target_critic_model

In [ ]:
her.train(1,1,100,1)

In [ ]:
wandb.finish()

In [ ]:
state = env.observation_space.sample()
state

In [ ]:
her.agent.state_normalizer.normalize(state)

In [ ]:
goal = her.desired_goal_func(her.agent.env)
goal

In [ ]:
her.agent.goal_normalizer.normalize(goal)

In [ ]:
def remove_renders(folder_path):
    # Iterate over the files in the folder
    for filename in os.listdir(folder_path):
        # Check if the file has a .mp4 or .meta.json extension
        if filename.endswith(".mp4") or filename.endswith(".meta.json"):
            # Construct the full file path
            file_path = os.path.join(folder_path, filename)
            # Remove the file
            os.remove(file_path)

In [ ]:
remove_renders("/workspaces/RL_Agents/pytorch/src/app/assets/models/ddpg/renders/training")

# HER Fetch-Reach (Robotics)

In [ ]:
env = gym.make("FetchReach-v2", max_episode_steps=50)

In [ ]:
desired_goal_func, achieved_goal_func, reward_func = gym_helper.get_her_goal_functions(env)

In [ ]:
achieved_goal_func(env)

In [ ]:
env.get_wrapper_attr("_get_obs")()

In [ ]:
# reset env state
env.reset()

In [ ]:
goal_shape = desired_goal_func(env).shape

In [ ]:
goal_shape

In [ ]:
# build actor

dense_layers = [
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    )
]

actor = models.ActorModel(env, cnn_model=None, dense_layers=dense_layers, goal_shape=goal_shape, optimizer='Adam',
                          optimizer_params={'weight_decay':0.0}, learning_rate=0.00001, normalize_layers=False)

In [ ]:
actor

In [ ]:
# build critic

state_layers = [
    
]

merged_layers = [
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
               
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
]


critic = models.CriticModel(env=env, cnn_model=None, state_layers=state_layers, merged_layers=merged_layers, goal_shape=goal_shape, optimizer="Adam", optimizer_params={'weight_decay':0.0}, learning_rate=0.00001, normalize_layers=False)

In [ ]:
critic

In [ ]:
replay_buffer = helper.ReplayBuffer(env, 1000000, goal_shape)
# noise = helper.OUNoise(shape=env.action_space.shape, dt=1.0, device='cuda')
noise = helper.NormalNoise(shape=env.action_space.shape, mean=0.0, stddev=0.05)

In [ ]:
ddpg_agent = rl_agents.DDPG(env=env,
                            actor_model=actor,
                            critic_model=critic,
                            discount=0.98,
                            tau=0.05,
                            action_epsilon=0.2,
                            replay_buffer=replay_buffer,
                            batch_size=256,
                            noise=noise,
                            callbacks=[rl_callbacks.WandbCallback("FetchReach-v2")])

In [ ]:
ddpg_agent.critic_model

In [ ]:
her = rl_agents.HER(
    agent=ddpg_agent,
    strategy='future',
    tolerance=0.05,
    num_goals=4,
    desired_goal=desired_goal_func,
    achieved_goal=achieved_goal_func,
    reward_fn=reward_func,
    normalizer_clip=5.0
)

In [ ]:
her.train(num_epochs=50,
          num_cycles=50,
          num_episodes=16,
          num_updates=40,
          render=True,
          render_freq=1000)

In [ ]:
states, action, rewards, next_states, dones, achieved_goals, next_achieved_goals, desired_goals = her.agent.replay_buffer.sample(2)

In [ ]:
desired_goals

In [ ]:
her.agent.env.get_wrapper_attr("distance_threshold")

In [ ]:
# get success
her.agent.env.get_wrapper_attr("_is_success")(achieved_goal_func(her.agent.env), desired_goal_func(her.agent.env))

In [ ]:
her.agent.env.get_wrapper_attr("goal_distance")(next_state_achieved_goal, desired_goal, None)

In [ ]:
pusher_her = rl_agents.HER.load("/workspaces/RL_Agents/pytorch/src/app/assets/models/her")

In [ ]:
pusher_her.agent.env.reset()

In [ ]:
pusher_her.get_config()

In [ ]:
wandb.finish()

In [ ]:
np.linalg.norm(pusher_her.agent.env.get_wrapper_attr("get_body_com")("goal") - pusher_her.agent.env.get_wrapper_attr("get_body_com")("object"))

In [ ]:
pusher_her.agent.replay_buffer.get_config()

In [ ]:

pusher_her.agent.replay_buffer.desired_goals

In [ ]:
## TEST ENV
env = gym.make("Pusher-v5", render_mode="rgb_array")

In [ ]:
env = gym.wrappers.RecordVideo(
                    env,
                    "/renders/training",
                    episode_trigger=lambda x: True,
                )


In [ ]:
state, _ = env.reset()

for i in range(1000):
# take action
    next_state, reward, term, trunc, _ = env.step(env.action_space.sample())
env.close()

# HER Fetch Push (Robitics)

In [ ]:
env = gym.make('FetchPush-v2')

In [ ]:
desired_goal_func, achieved_goal_func, reward_func = gym_helper.get_her_goal_functions(env)

In [ ]:
# reset env state
env.reset()

In [ ]:
goal_shape = desired_goal_func(env).shape

In [ ]:
# build actor

dense_layers = [
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    )
]

actor = models.ActorModel(env, cnn_model=None, dense_layers=dense_layers, goal_shape=goal_shape, optimizer='Adam',
                          optimizer_params={'weight_decay':0.0}, learning_rate=0.00001, normalize_layers=False)

In [ ]:
# build critic

state_layers = [
    
]

merged_layers = [
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
               
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
]


critic = models.CriticModel(env=env, cnn_model=None, state_layers=state_layers, merged_layers=merged_layers, goal_shape=goal_shape, optimizer="Adam", optimizer_params={'weight_decay':0.0}, learning_rate=0.00001, normalize_layers=False)

In [ ]:
replay_buffer = helper.ReplayBuffer(env, 1000000, goal_shape)
# noise = helper.OUNoise(shape=env.action_space.shape, dt=1.0, device='cuda')
noise = helper.NormalNoise(shape=env.action_space.shape, mean=0.0, stddev=0.05)

In [ ]:
ddpg_agent = rl_agents.DDPG(env=env,
                            actor_model=actor,
                            critic_model=critic,
                            discount=0.98,
                            tau=0.05,
                            action_epsilon=0.3,
                            replay_buffer=replay_buffer,
                            batch_size=128,
                            noise=noise,
                            callbacks=[rl_callbacks.WandbCallback("FetchPush-v2")],
                            save_dir="fetch_push/models/ddpg/"
                            )

In [ ]:
her = rl_agents.HER(
    agent=ddpg_agent,
    strategy='final',
    tolerance=0.05,
    num_goals=4,
    desired_goal=desired_goal_func,
    achieved_goal=achieved_goal_func,
    reward_fn=reward_func,
    normalizer_clip=5.0,
    save_dir="fetch_push/models/her/"
)

In [ ]:
her.train(num_epochs=50,
          num_cycles=50,
          num_episodes=16,
          num_updates=40,
          render=True,
          render_freq=1000)

# TESTING MULTITHREADING

In [ ]:
env = gym.make('FetchPush-v2')

In [ ]:
desired_goal_func, achieved_goal_func, reward_func = gym_helper.get_her_goal_functions(env)

In [ ]:
# reset env state
env.reset()

In [ ]:
goal_shape = desired_goal_func(env).shape

In [ ]:
# build actor

dense_layers = [
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    )
]

actor = models.ActorModel(env, cnn_model=None, dense_layers=dense_layers, goal_shape=goal_shape, optimizer='Adam',
                          optimizer_params={'weight_decay':0.0}, learning_rate=0.00001, normalize_layers=False)

In [ ]:
# build critic

state_layers = [
    
]

merged_layers = [
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
               
            }
        },
    ),
    (
        64,
        "relu",
        {
            "kaiming uniform": {
                
            }
        },
    ),
]


critic = models.CriticModel(env=env, cnn_model=None, state_layers=state_layers, merged_layers=merged_layers, goal_shape=goal_shape, optimizer="Adam", optimizer_params={'weight_decay':0.0}, learning_rate=0.00001, normalize_layers=False)

In [ ]:
replay_buffer = helper.ReplayBuffer(env, 1000000, goal_shape)
# noise = helper.OUNoise(shape=env.action_space.shape, dt=1.0, device='cuda')
noise = helper.NormalNoise(shape=env.action_space.shape, mean=0.0, stddev=0.05)

In [ ]:
ddpg_agent = rl_agents.DDPG(env=env,
                            actor_model=actor,
                            critic_model=critic,
                            discount=0.98,
                            tau=0.05,
                            action_epsilon=0.3,
                            replay_buffer=replay_buffer,
                            batch_size=128,
                            noise=noise,
                            callbacks=[rl_callbacks.WandbCallback("FetchPush-v2")],
                            save_dir="fetch_push/models/ddpg/"
                            )

In [ ]:
her = rl_agents.HER(
    agent=ddpg_agent,
    strategy='final',
    num_workers=4,
    tolerance=0.05,
    num_goals=4,
    desired_goal=desired_goal_func,
    achieved_goal=achieved_goal_func,
    reward_fn=reward_func,
    normalizer_clip=5.0,
    save_dir="fetch_push/models/her/"
)

In [ ]:
her.train()

# TESTING

In [ ]:
# load config
config_path = "/workspaces/RL_Agents/pytorch/src/app/HER_Test/her/config.json"
with open(config_path, 'r') as file:
    config = json.load(file)

In [ ]:
config

In [ ]:
agent = rl_agents.HER.load(config)

In [ ]:
for callback in agent.agent.callbacks:
    print(callback._sweep)

# Co Occurence

In [ ]:
import subprocess

In [9]:
# Define the path to your JSON configuration file
config_file_path = 'assets/wandb_config.json'

# Read the JSON configuration file
with open(config_file_path, 'r') as file:
    wandb_config = json.load(file)

# Print the configuration to verify it has been loaded correctly
print(wandb_config)

{'method': 'bayes', 'project': 'FetchReach-v2', 'name': 'Test', 'metric': {'name': 'episode_reward', 'goal': 'maximize'}, 'parameters': {'env': {'parameters': {'id': {'value': 'FetchReach-v2'}, 'max_episode_steps': {'value': 50}}}, 'model_type': {'values': ['HER_DDPG']}, 'HER_DDPG': {'parameters': {'HER_DDPG_actor_learning_rate': {'values': [1e-05]}, 'HER_DDPG_critic_learning_rate': {'values': [1e-05]}, 'HER_DDPG_goal_strategy': {'values': ['future']}, 'HER_DDPG_num_goals': {'min': 4, 'max': 8}, 'HER_DDPG_goal_tolerance': {'values': [0.05]}, 'HER_DDPG_discount': {'values': [0.99]}, 'HER_DDPG_tau': {'values': [0.05]}, 'HER_DDPG_epsilon_greedy': {'values': [0.3]}, 'HER_DDPG_normalizer_clip': {'values': [5]}, 'HER_DDPG_device': {'value': 'cuda'}, 'HER_DDPG_actor_num_cnn_layers': {'value': 0}, 'HER_DDPG_actor_num_layers': {'value': 2}, 'HER_DDPG_actor_activation': {'values': ['relu']}, 'HER_DDPG_actor_hidden_kernel_initializer': {'values': ['kaiming_uniform']}, 'HER_DDPG_actor_output_kerne

In [10]:
# Define the path to your JSON configuration file
config_file_path = 'assets/sweep_config.json'

# Read the JSON configuration file
with open(config_file_path, 'r') as file:
    sweep_config = json.load(file)

# Print the configuration to verify it has been loaded correctly
print(sweep_config)

{'num_sweeps': 5, 'num_episodes': 10, 'seed': 42, 'use_mpi': False, 'num_workers': None, 'num_agents': None, 'num_epochs': 1, 'num_cycles': 1, 'num_updates': None}


In [11]:
# Save the updated configuration to a train config file
os.makedirs('sweep', exist_ok=True)
train_config_path = os.path.join(os.getcwd(), 'sweep/train_config.json')
with open(train_config_path, 'w') as f:
    json.dump(sweep_config, f)

# Save and Set the sweep config path
sweep_config_path = os.path.join(os.getcwd(), 'sweep/sweep_config.json')
with open(sweep_config_path, 'w') as f:
    json.dump(wandb_config, f)

In [ ]:
command = ['python', 'sweep.py']

# Set the environment variable
os.environ['WANDB_DISABLE_SERVICE'] = 'true'

subprocess.Popen(command)

In [ ]:
# Set the environment variable
os.environ['WANDB_DISABLE_SERVICE'] = 'true'

In [2]:
# Define the path to your JSON configuration file
config_file_path = 'sweep/sweep_config.json'

# Read the JSON configuration file
with open(config_file_path, 'r') as file:
    sweep_config = json.load(file)

# Print the configuration to verify it has been loaded correctly
print(sweep_config)

{'method': 'bayes', 'project': 'FetchReach-v2', 'name': 'Test', 'metric': {'name': 'episode_reward', 'goal': 'maximize'}, 'parameters': {'env': {'parameters': {'id': {'value': 'FetchReach-v2'}, 'max_episode_steps': {'value': 50}}}, 'model_type': {'values': ['HER_DDPG']}, 'HER_DDPG': {'parameters': {'HER_DDPG_actor_learning_rate': {'values': [1e-05, 0.0001]}, 'HER_DDPG_critic_learning_rate': {'values': [1e-05, 0.0001]}, 'HER_DDPG_goal_strategy': {'values': ['future']}, 'HER_DDPG_num_goals': {'min': 4, 'max': 8}, 'HER_DDPG_goal_tolerance': {'values': [0.05]}, 'HER_DDPG_discount': {'values': [0.9, 0.99]}, 'HER_DDPG_tau': {'values': [0.05]}, 'HER_DDPG_epsilon_greedy': {'values': [0.2, 0.3]}, 'HER_DDPG_normalizer_clip': {'values': [5]}, 'HER_DDPG_device': {'value': 'cuda'}, 'HER_DDPG_actor_num_cnn_layers': {'value': 0}, 'HER_DDPG_actor_num_layers': {'value': 2}, 'HER_DDPG_actor_activation': {'values': ['relu']}, 'HER_DDPG_actor_hidden_kernel_initializer': {'values': ['kaiming_uniform']}, 'H

In [3]:
# Define the path to your JSON configuration file
config_file_path = 'sweep/train_config.json'

# Read the JSON configuration file
with open(config_file_path, 'r') as file:
    train_config = json.load(file)

# Print the configuration to verify it has been loaded correctly
print(train_config)

{'num_sweeps': 5, 'num_episodes': 10, 'seed': 42, 'use_mpi': False, 'num_workers': 1, 'num_agents': 1, 'num_epochs': 1, 'num_cycles': 1, 'num_updates': 1}


In [4]:
from dash_callbacks import run_agent

sweep_id = wandb.sweep(sweep=sweep_config, project=sweep_config["project"])
# loop over num wandb agents
num_agents = 2
for agent in range(num_agents):
    
)

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.wandb.ai:443
DEBUG:urllib3.connectionpool:https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 189
INFO:wandb.agents.pyagent:Starting sweep agent: entity=None, project=FetchReach-v2, count=5
DEBUG:wandb.agents.pyagent:Agent._setup()
DEBUG:wandb.agents.pyagent:Agent._register()
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.wandb.ai:443
DEBUG:urllib3.connectionpool:https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 68
DEBUG:wandb.agents.pyagent:agent_id = QWdlbnQ6ajh4dmtpczg=


Create sweep with ID: 5vujbkhg
Sweep URL: https://wandb.ai/jasonhayes1987/FetchReach-v2/sweeps/5vujbkhg


DEBUG:urllib3.connectionpool:https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 808
DEBUG:wandb.agents.pyagent:Job received: Job(lqtsr2zi,{'HER_DDPG': {'value': {'HER_DDPG_actor_activation': 'relu', 'HER_DDPG_actor_clamp_output': 0.04, 'HER_DDPG_actor_hidden_kernel_initializer': 'kaiming_uniform', 'HER_DDPG_actor_hidden_kernel_kaiming_uniform': {'kaiming_uniform_mode': 'fan_in'}, 'HER_DDPG_actor_learning_rate': 1e-05, 'HER_DDPG_actor_normalize_layers': False, 'HER_DDPG_actor_num_cnn_layers': 0, 'HER_DDPG_actor_num_layers': 2, 'HER_DDPG_actor_optimizer': 'Adam', 'HER_DDPG_actor_optimizer_Adam_options': {'Adam_weight_decay': 0}, 'HER_DDPG_actor_output_kernel_constant': {'constant_value': 0.003}, 'HER_DDPG_actor_output_kernel_initializer': 'constant', 'HER_DDPG_batch_size': 128, 'HER_DDPG_critic_activation': 'relu', 'HER_DDPG_critic_hidden_kernel_initializer': 'kaiming_uniform', 'HER_DDPG_critic_hidden_kernel_kaiming_uniform': {'kaiming_uniform_mode': 'fan_in'}, 'HER_DDPG_critic_learni

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.wandb.ai:443


her model creation fired
sweep not using mpi fired
wandb config:{'HER_DDPG': {'HER_DDPG_actor_activation': 'relu', 'HER_DDPG_actor_clamp_output': 0.04, 'HER_DDPG_actor_hidden_kernel_initializer': 'kaiming_uniform', 'HER_DDPG_actor_hidden_kernel_kaiming_uniform': {'kaiming_uniform_mode': 'fan_in'}, 'HER_DDPG_actor_learning_rate': 1e-05, 'HER_DDPG_actor_normalize_layers': False, 'HER_DDPG_actor_num_cnn_layers': 0, 'HER_DDPG_actor_num_layers': 2, 'HER_DDPG_actor_optimizer': 'Adam', 'HER_DDPG_actor_optimizer_Adam_options': {'Adam_weight_decay': 0}, 'HER_DDPG_actor_output_kernel_constant': {'constant_value': 0.003}, 'HER_DDPG_actor_output_kernel_initializer': 'constant', 'HER_DDPG_batch_size': 128, 'HER_DDPG_critic_activation': 'relu', 'HER_DDPG_critic_hidden_kernel_initializer': 'kaiming_uniform', 'HER_DDPG_critic_hidden_kernel_kaiming_uniform': {'kaiming_uniform_mode': 'fan_in'}, 'HER_DDPG_critic_learning_rate': 1e-05, 'HER_DDPG_critic_merged_num_layers': 2, 'HER_DDPG_critic_normalize_lay

...

wandb init called
wandb run tag complete
env spec: EnvSpec(id='FetchReach-v2', entry_point='gymnasium_robotics.envs.fetch.reach:MujocoFetchReachEnv', reward_threshold=None, nondeterministic=False, max_episode_steps=50, order_enforce=True, disable_env_checker=False, kwargs={'reward_type': 'sparse'}, namespace=None, name='FetchReach', version=2, additional_wrappers=(), vector_entry_point=None)
save dir set:HER_Test
if wandb run fired
agent name: ddpg
new save dir: HER_Test/her/ddpg/
self._obs_space_shape: (10,)
buffer size = 100000
agent built:{'agent_type': 'HER', 'agent': {'agent_type': 'DDPG', 'env': 'FetchReach-v2', 'actor_model': {'env': 'FetchReach-v2', 'cnn_model': None, 'num_layers': 4, 'dense_layers': [(128, 'relu', {'kaiming_uniform': {'mode': 'fan_in'}}), (64, 'relu', {'kaiming_uniform': {'mode': 'fan_in'}})], 'output_layer_kernel': {'constant': {'val': 0.003}}, 'goal_shape': (3,), 'optimizer': 'Adam', 'optimizer_params': {'weight_decay': 0}, 'learning_rate': 1e-05, 'normalize

wandb: Adding directory to artifact (./HER_Test/her/ddpg)... Done. 0.0s
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.wandb.ai:443
DEBUG:urllib3.connectionpool:https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 42
DEBUG:urllib3.connectionpool:https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 57


epoch 0 cycle 1 episode 10, success percentage 0.0, reward -50.0, avg reward -49.2, avg episode time 0.15s


DEBUG:urllib3.connectionpool:https://api.wandb.ai:443 "POST /graphql HTTP/1.1" 200 57


action_0,▄▆▄▄▅▁▁▆▄▅▆▃▄▅▄▄▅▄▄▅▃▆▇▄▄▄▅▅▄▄▅▄▅▅▂▄█▅▅▄
action_1,▄▇▄▄▄▁▄▇▄▅▆▅▄▃▄▄▄▂▄▄▄▇▆▄▄▄▄█▄▄▄▄▄▄▅▄▃▄▄▄
action_2,▄█▄▄▄▁▇▄▄▄▃▁▄▅▄▄▄▁▄▄█▂█▄▅▄▄▇▄▄▄▄▄▄█▄▆▄▄▄
action_3,▄▆▅▅▅▁▃▆▅▅▆▅▅▅▅▄▅▆▅▅▇▄▁▅▅▅▅█▅▅▅▅▅▅▂▅▄▅▅▅
actor_loss,▁▂▃▃▄▅▆▆▇█
actor_predictions,█████▃▃▃▃▄▄▄▄▂▂▂▂▂▃▃▃▃▆▆▆▆▆▄▄▄▄█████▁▁▁▁
avg_reward,▁█▆▅▄▃▃▃▃▂
best,██▁▁▁▁▁▁▁▁
critic_loss,▄▁▄▄▃▃▃█▃█
critic_predictions,█████▇▇▇▇▆▆▆▆▅▅▅▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▂▂▂▂▂▁▁▁▁
episode,▁▂▃▃▄▅▆▆▇█


In [ ]:
sweep_config

In [12]:
env = gym.make("FetchReach-v2")

In [22]:
env_spec = env.spec.to_json()

In [24]:
env = gym.make(gym.envs.registration.EnvSpec.from_json(env_spec))

In [26]:
env.spec

EnvSpec(id='FetchReach-v2', entry_point='gymnasium_robotics.envs.fetch.reach:MujocoFetchReachEnv', reward_threshold=None, nondeterministic=False, max_episode_steps=50, order_enforce=True, disable_env_checker=False, kwargs={'reward_type': 'sparse'}, namespace=None, name='FetchReach', version=2, additional_wrappers=(), vector_entry_point=None)

In [18]:
def load_env_spec(env_spec_dict):
    # Create a new EnvSpec instance using the dictionary
    env_spec = gym.envs.registration.EnvSpec(**env_spec_dict)
    return env_spec

In [19]:
load_env_spec(env_spec)

TypeError: EnvSpec.__init__() got an unexpected keyword argument 'namespace'